[Poprzedni rozdzial](rozdzial_03_black_scholes.ipynb)


# 4. Ustawienia eksperymentów

In [1]:
from pathlib import Path
import pickle
from IPython.display import HTML, display

ROOT = Path.cwd().resolve()
while not (ROOT / "parameters.py").exists() and ROOT != ROOT.parent:
    ROOT = ROOT.parent

def _html_table(table):
    header = "".join([f"<th>{h}</th>" for h in table[0]])
    rows = []
    for row in table[1:]:
        cells = "".join([f"<td>{c}</td>" for c in row])
        rows.append(f"<tr>{cells}</tr>")
    table_html = "<table>" + "<tr>" + header + "</tr>" + "".join(rows) + "</table>"
    return table_html

PLOTS_DIR = ROOT / "plots"


## 4.1. Liczba symulacji R

Dla estymatora  
$$
\hat{I} = \frac{1}{R}\sum_{i=1}^R Y_i
$$  
błąd standardowy wynosi  
$$
\mathrm{SE}(\hat{I}) = \frac{\sigma}{\sqrt{R}},
$$  
czyli dokładność rośnie powoli (proporcjonalnie do $1/\sqrt{R}$). Dobór $R$ musi więc równoważyć precyzję i koszt obliczeń.

W profesjonalnej wycenie opcji Monte Carlo używa się zwykle $10^4\!-\!10^5$ symulacji dla instrumentów o umiarkowanej wariancji wypłaty. Taki rząd wielkości zapewnia stabilność wyników i błędy standardowe na poziomie akceptowalnym w praktyce rynkowej.

W projekcie przyjęliśmy  
$$
R = 100{,}000,
$$  
co mieści się w typowym zakresie stosowanym w finansach i gwarantuje wiarygodne oszacowania przy rozsądnym czasie obliczeń. W metodach redukcji wariancji taka wartość jest nawet nadmiarowa, ponieważ ich estymatory mają mniejszą wariancję.

Warto zauważyć, że wartość $R = 100{,}000$, dobrana pod kątem opcji europejskiej (czyli bez bariery), jest tym bardziej wystarczająca w przypadku opcji z barierą. Jeśli oznaczymy wypłatę opcji europejskiej jako $A_\infty$ oraz wypłatę opcji z barierą $C$ jako $A_C$, to zachodzi  
$$
A_C = A_\infty \cdot \mathbf{1}_{\{\text{bariera nie została trafiona}\}},
$$  
czyli $A_C \le A_\infty$ dla każdej realizacji. Ponadto  
$$
A_C^2 \le A_\infty^2 \quad \text{oraz} \quad \mathbb{E}[A_C^2] < \mathbb{E}[A_\infty^2],
$$  
więc wariancja opcji barierowej spełnia  
$$
\mathrm{Var}(A_C) = \mathbb{E}[A_C^2] - \mathbb{E}[A_C]^2 < \mathbb{E}[A_\infty^2] - \mathbb{E}[A_\infty]^2 = \mathrm{Var}(A_\infty).
$$  
Zatem wariancja wypłat (czyli $\sigma^2$ używana w oszacowaniu błędu standardowego) jest mniejsza w przypadku opcji z barierą, co oznacza, że błąd standardowy $\mathrm{SE}(\hat{I})$ również jest mniejszy niż dla opcji europejskiej — przy tej samej liczbie symulacji $R$. Przyjęta wartość $R = 100{,}000$ jest więc tym bardziej wystarczająca dla przypadku z barierą.


## 4.2 Dyskretyzacja osi czasu (n)

Symulacje GBM obserwujemy w punktach
$$
t_k = \frac{k}{n},
$$
więc $n$ jest parametrem dyskretyzacji wpływającym na wykrywanie przekroczeń bariery.
Większe $n$ oznacza dokładniejszą obserwację ścieżki, ale też wyższy koszt obliczeń
(złożoność $\sim R \cdot n$).

W praktycznej wycenie opcji barierowych **nie stosuje się bariery ciągłej** — kontrakt
określa konkretne daty monitorowania, zwykle dzienne (ok. 250/rok), tygodniowe (ok. 52/rok)
lub miesięczne (ok. 12/rok). Profesjonalne modele odzwierciedlają właśnie taką dyskretną
strukturę, a nie idealizację ciągłą.

Traktując $T = 1$ jako rok, przyjęcie
$$
n = 12
$$
jest zgodne z **typową częstotliwością miesięczną** i odpowiada praktycznym standardom
wyceny dyskretnych barier, zapewniając jednocześnie rozsądny koszt obliczeń.

## 4.3. Wartości barier $C_1$ i $C_2$

Wybór dwóch wartości bariery up-and-out — $C_1 = 105$ oraz $C_2 = 120$ — pozwala analizować dwa odmienne reżimy zachowania opcji barierowej przy założeniu $S_0 = 100$.

Poziom $C_1 = 105$ reprezentuje barierę bardzo bliską cenie początkowej, co skutkuje wysokim prawdopodobieństwem knock-outu i silną zależnością wyniku od maksymalnych wartości ścieżki. Taka konfiguracja jest szczególnie interesująca numerycznie, ponieważ ujawnia problemy związane z dyskretnym monitoringiem bariery oraz konieczność stosowania korekt, takich jak Brownian Bridge. Jednocześnie stanowi klasyczny przykład „silnie barierowy”, często wykorzystywany w literaturze do testowania jakości metod Monte Carlo.

Z kolei bariera $C_2 = 120$ znajduje się znacznie dalej, co prowadzi do znacznie niższego prawdopodobieństwa aktywacji bariery i sprawia, że opcja zachowuje się bardziej jak instrument vanilla. Dzięki temu przypadek ten jest stabilniejszy numerycznie i pozwala badać, jak zmienia się wycena oraz charakterystyki opcji, gdy bariera ma marginalny wpływ na dynamikę wypłaty. Wspólne rozważenie obu poziomów umożliwia uchwycenie przejścia od silnego do słabego efektu bariery, dając pełniejszy obraz zjawiska i zgodnie z praktyką literaturową tworzy reprezentatywny zestaw przykładów do analiz teoretycznych i numerycznych.

## 4.4. Liczba strat $m$

W metodzie stratyfikacji ważne jest, aby w każdej stracie znajdowała się wystarczająco duża liczba próbek. Jeżeli całkowita liczba symulacji to $R$, to liczba próbek przypadających na jedną stratę wynosi

$$
R_i = \frac{R}{m}.
$$

Gdy $m$ jest zbyt duże, wtedy $R_i$ staje się małe, a średnie warunkowe $\bar{Y}_i$ obliczane w poszczególnych stratach mają wysoką wariancję. W skrajnym przypadku, kiedy $m$ byłoby zbliżone do $R$, każda strata miałaby około jednej próbki, przez co stratyfikacja przestaje działać i sprowadza się do zwykłego Monte Carlo.

W celu zbadania wpływu liczby stratum $m$ na jakość estymacji w metodzie stratyfikacji
rozważamy trzy wartości:
$$
m \in \{10, 20, 50\}.
$$
Dla każdej z nich analizujemy dwa schematy alokacji próby: **alokację proporcjonalną**
oraz **alokację optymalną**. Całkowita liczba symulacji $R$ jest stała, a dla każdej
kombinacji liczby stratum i schematu alokacji wykonujemy **100 niezależnych powtórzeń
estymatora**. Dla każdego powtórzenia obliczamy błąd estymacji jako różnicę pomiędzy
wartością uzyskaną metodą Monte Carlo a dokładną wartością wyznaczoną ze wzoru
Blacka--Scholesa.

In [2]:
hist_path = PLOTS_DIR / "strata_errors_hist.pkl"
box_path = PLOTS_DIR / "strata_errors_box.pkl"

with hist_path.open("rb") as f:
    hist_fig = pickle.load(f)
hist_fig

In [5]:
with box_path.open("rb") as f:
    box_fig = pickle.load(f)
box_fig

In [6]:
opt_hist_path = PLOTS_DIR / "strata_errors_hist_optimal.pkl"
opt_box_path = PLOTS_DIR / "strata_errors_box_optimal.pkl"

with opt_hist_path.open("rb") as f:
    opt_hist_fig = pickle.load(f)
opt_hist_fig

In [7]:
with opt_box_path.open("rb") as f:
    opt_box_fig = pickle.load(f)
opt_box_fig

Na wykresach przedstawiono rozkłady błędów estymacji w postaci histogramów oraz
wykresów pudełkowych. Pierwsze dwa wykresy odpowiadają estymatorom z
**alokacją proporcjonalną**, natomiast kolejne dwa --- estymatorom z
**alokacją optymalną**.

W przypadku alokacji proporcjonalnej przejście z $m=10$ do $m=20$ prowadzi do
widocznego zmniejszenia rozrzutu błędów estymacji. Dalsze zwiększenie liczby stratum
do $m=50$ nie powoduje jednak wyraźnej dodatkowej poprawy --- rozkłady błędów dla
$m=20$ i $m=50$ są zbliżone zarówno pod względem szerokości, jak i położenia
względem zera.

Dla alokacji optymalnej obserwujemy podobny trend. Wartość $m=20$ prowadzi do
wyraźnie skoncentrowanego rozkładu błędów wokół zera. W przypadku $m=50$ rozrzut
błędów pozostaje porównywalny, a w szczególności nie obserwujemy pogorszenia jakości
estymacji. Co więcej, w porównaniu z $m=20$ widoczny jest brak najbardziej skrajnych
obserwacji odstających, co może sugerować nieco większą stabilność rozkładu błędów,
choć różnice te są niewielkie.

Na podstawie przeprowadzonych symulacji można stwierdzić, że zwiększanie liczby
stratum powyżej $m=20$ nie przynosi istotnych korzyści w kontekście redukcji błędu
estymacji, zarówno dla alokacji proporcjonalnej, jak i optymalnej. Jednocześnie nie ma
jednoznacznych przesłanek, aby uznać wybór $m=50$ za gorszy --- obserwowane różnice
pomiędzy $m=20$ a $m=50$ są raczej drugorzędne.

W związku z tym wybór $m=20$ można traktować jako rozsądny kompromis pomiędzy
szczegółowością podziału na straty a stabilnością estymatora przy danym budżecie
symulacyjnym. Wyniki sugerują jednak, że przy zastosowaniu alokacji optymalnej
również większa liczba stratum może być akceptowalna, choć potencjalne zyski są
ograniczone.

## 4.5. Liczba powtórzeń eksperymentu $M$

W celu empirycznej oceny własności statystycznych analizowanych estymatorów każdy eksperyment Monte Carlo został powtórzony $M = 1000$ razy. Taka liczba replikacji pozwala na stabilne oszacowanie wariancji estymatorów, błędu standardowego oraz współczynnika redukcji wariancji (VRF), a także umożliwia wiarygodną ocenę rozkładu estymat.

Przy $M = 1000$ replikacjach błąd estymacji empirycznej wariancji jest relatywnie mały, co zapewnia porównywalność wyników między metodami i ogranicza wpływ losowych fluktuacji. Jednocześnie wartość ta stanowi rozsądny kompromis pomiędzy dokładnością wnioskowania statystycznego a kosztem obliczeniowym całego eksperymentu.

[Nastepny rozdzial](rozdzial_05_wyniki.ipynb)
